In [1]:
import sys
import numpy as np
import pandas as pd
import os.path as osp
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.externals import joblib
sys.path.append("../")
from parse_fasta import *
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [32]:
# Load data
dataset_url = osp.join('..', 'dataset', 'subloc', 'eukaryota', 'subloc_k3_s5_eukaryota.1682.fa_1516057206692442.csv')
df = pd.read_csv(dataset_url, sep=',')
print(df.shape)
df[:15]

(1673, 8002)


,label,id,AAA,AAR,AAN,AAD,AAC,AAQ,AAE,AAG,...,VVL,VVK,VVM,VVF,VVP,VVS,VVT,VVW,VVY,VVV
0,cytoplasm,RS23_YEAST,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,memmitochondria,ATP8_PISOC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,secreted,GOME_ACAGO,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,chloroplast,RK21_EMIHU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,memmitochondria,NU4LM_BOVIN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,secreted,THMAC_THETS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,secreted,SGP1_SCHGR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,secreted,NDB43_OPICY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,secreted,LEC_ALOAR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,chloroplast,YCXG_PORPU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [111]:
# transform labels to numerical
y = df['label'].values
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)

In [112]:
le.classes_
np.unique(y, return_counts=True)

(array([0, 1]), array([13, 41]))

In [62]:
thresh = 10
_, counts = np.unique(y, return_counts=True)
print(np.sum(counts[counts < 10]) == len(df) - len(df[mask]))
mask = np.isin(y, np.where(counts >= thresh)[0])
df[mask]

True


,label,AAA,AAR,AAN,AAD,AAC,AAQ,AAE,AAG,AAH,...,VVL,VVK,VVM,VVF,VVP,VVS,VVT,VVW,VVY,VVV
id,,,,,,,,,,,,,,,,,,,,,
STP4_PIG,nucleus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
YL87_SCHPO,memmitochondria,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
MDHC_CANAL,cytoplasm,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
FRDA_TRAHO,cytoplasm,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
PIMP1_CAPAN,cellmembrane,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
IAH1_SCHPO,cytoplasm,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
APOA2_MOUSE,secreted,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
DPOD4_HUMAN,nucleus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
NEUFC_MOUSE,secreted,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
for cls in l:
    np.where(y == cls)

In [64]:
# Split data into training and test sets
X = data.drop('class', axis=1)
# split to train-test
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.2, 
                                                    random_state=123)

#X_train, X_test, y_train, y_test = train_test_split(X, y, 
 #                                                   test_size=0.2, 
  #                                                  random_state=123,
   #                                                 stratify=y)

In [65]:
# Declare data preprocessing steps
pipeline = make_pipeline(preprocessing.StandardScaler(), 
                         RandomForestClassifier(n_estimators=50))

In [66]:
# Declare hyperparameters to tune
hyperparameters = { 'randomforestclassifier__max_features' : [None, 'sqrt', 'log2'],
                  'randomforestclassifier__max_depth': [None]}

In [67]:
# Tune model using cross-validation pipeline
clf = GridSearchCV(pipeline, hyperparameters, cv=5, verbose=1, n_jobs=64, refit='accuracy', scoring='accuracy')

In [68]:
clf.fit(X_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


/home/olle/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:597: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=64)]: Done  15 out of  15 | elapsed:    0.7s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('randomforestclassifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid=True, n_jobs=64,
       param_grid={'randomforestclassifier__max_features': [None, 'sqrt', 'log2'], 'randomforestclassifier__max_depth': [None]},
       pre_dispatch='2*n_jobs', refit='accuracy', return_train_score=True,
       scoring='accuracy', verbose=1)

In [69]:
best_estimator = clf.best_estimator_

In [70]:
# Evaluate model pipeline on test data
pred = best_estimator.predict(X_test)
print(accuracy_score(y_test, pred))
print(mean_squared_error(y_test, pred))
print(best_estimator.get_params(deep=False))

0.444444444444
8.33333333333
{'memory': None, 'steps': [('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('randomforestclassifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]}


In [71]:
# Save model for future use
joblib.dump(clf, 'rf_model_03.pkl')

['rf_model_03.pkl']

In [73]:
# Load model
clf2 = joblib.load('rf_model_03.pkl')

In [88]:
unique, counts = np.unique(y_test, return_counts=True)

#computing uniformly (random guessing) class predictions
n_samples = len(y_test)
n_classes = len(unique)
uniform_preds = np.random.randint(n_classes, size=n_samples)

#computing class predictions using Naive bayes(i.e. class probs from data)
class_probs = counts/n_samples
naive_preds = np.random.choice(unique, size=n_samples, p=class_probs)

In [89]:
# Evaluate model pipeline on test data
pred = clf2.predict(X_test)
print('Acurracy of random guessing: {}')
print(accuracy_score(y_test, uniform_preds))
print('Naive Bayes accuracy: {}' )
print(accuracy_score(y_test, naive_preds))
print('Accuracy of best model: {}')
print(accuracy_score(y_test, pred))

print(mean_squared_error(y_test, pred))
print(clf2.get_params(deep=False))

Acurracy of random guessing: {}
0.0
Naive Bayes accuracy: {}
0.555555555556
Accuracy of best model: {}
0.444444444444
8.33333333333
{'cv': 5, 'error_score': 'raise', 'estimator': Pipeline(memory=None,
     steps=[('standardscaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('randomforestclassifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]), 'fit_params': None, 'iid': True, 'n_jobs': 64, 'param_grid': {'randomforestclassifier__max_features': [None, 'sqrt', 'log2'], 'randomforestclassifier__max_depth': [None]}, 'pre_dispatch': '2*n_jobs', 'refit': 'accuracy', 'return_train_score': True, 'scoring': 'accuracy', 'verbose': 1}
